<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Current Todo:


Some ideas here: 

* Use pandas_ta to attach technical data to timeseries then use random forest to see if any features are super good.

* Use vector_bt to back test your models

* Integrate the different time granularity models, 15m etc into the core configs

* Fix coinbase - we can't filter on volume amount without extra work

* Add TA fields, use random forest to check which fields are the best, verify with mean square error (this could take a while)

* Move to an "always" on model where we more closely monitor exits and entries. We can constantly check predictions and refine entry/exit points. Can we pick a better entry point? We had at some point discussed trying to predict all 4 values which would help. 

* Can we monitor and alarm/email the 15 min chart of an entry and see if it is
move toward or away from our target?

* pandas_ta strategy/back testers look interesting?

Read me: https://www.kaggle.com/code/vuhuyduongnia/vn30-stock-prediction-by-lstm-model-accuracy-90


#IMPORT DATASETS AND LIBRARIES


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import sys

sys.path.insert(0,'/content/drive/My Drive/ml-trde-notebooks')




Mounted at /content/drive


# Library

In [ ]:
# ***** WARNING : Install deps - This will BUILD TALib and takes a while!
%run -i '/content/drive/My Drive/ml-trde-notebooks/installs.ipynb'

--2023-01-28 00:02:49--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-01-28 00:02:49--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://onboardcloud.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-01-28 00:02:50--  http://onboardcloud.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving onboa

In [ ]:
# Executes this notebook in our space, making all of its functions/globals available
%run -i '/content/drive/My Drive/ml-trde-notebooks/common.ipynb'
train_models = False
save_models = False
load_models = True


Modified
1674857475
1674857475.0
1674857475.0


# Load all models from disk

In [ ]:
if load_models:
  load_all_models()
else:
  print ("Not loading")
  


loading: lstm_cv
loading: lstm_coins_cv
loading: lstm_att_cv
loading: lstm_att_ohlcv
loading: lstm_cv_rvi
loading: lstm_cv_vwap
loading: lstm_ohlc
loading: svm_cv
loading: svm_cv_vwap
loading: lstm_xgb_cols
models loaded


# Visualize and Compare all Models

In [ ]:
df_raw = get_coin_data_frames(180, "BTC-USDT")

In [ ]:
backtest_filter = []
[results, data, features, fig] = renderPredictions(df_raw, models, [], False, False)
features[["Date", "Close", "Target", "Predicted"]].tail(1)

key: lstm_cv
testing: lstm_cv
6/6 [==============================] - 0s 3ms/step
predicted mse for model:  lstm_cv 0.004158250568930274
profit for model:  lstm_cv 8042.045555904072
key: lstm_coins_cv
testing: lstm_coins_cv
6/6 [==============================] - 0s 3ms/step
predicted mse for model:  lstm_coins_cv 0.004337444022059591
profit for model:  lstm_coins_cv 8051.295360343454
key: lstm_att_cv
testing: lstm_att_cv
6/6 [==============================] - 0s 3ms/step
predicted mse for model:  lstm_att_cv 0.004105615956083516
profit for model:  lstm_att_cv 7978.490451387806
key: lstm_att_ohlcv
testing: lstm_att_ohlcv
6/6 [==============================] - 0s 3ms/step
predicted mse for model:  lstm_att_ohlcv 0.004212291575887523
profit for model:  lstm_att_ohlcv 7166.286124717044
key: lstm_cv_rvi
testing: lstm_cv_rvi
6/6 [==============================] - 0s 3ms/step
predicted mse for model:  lstm_cv_rvi 0.004320399249131824
profit for model:  lstm_cv_rvi 8020.483947158291
key: lstm_c

,Date,Close,Target,Predicted
179,2023-01-28,23060.400000,23060.400000,22968.587343


In [ ]:
results

,0,1,2
0,lstm_cv,0.002103,13410.077348
0,lstm_coins_cv,0.002184,13502.825211
0,lstm_att_cv,0.002072,13541.177308
0,lstm_att_ohlcv,0.002401,12436.162455
0,lstm_cv_rvi,0.002240,13500.771401
0,lstm_cv_vwap,0.002212,13491.299957
0,lstm_ohlc,0.002390,12840.156185
0,svm_cv,0.002116,13541.177308
0,svm_cv_vwap,0.002138,13495.717028
0,lstm_xgb_cols,0.002157,13207.700448


# What has a buy indicator for tomorrow?

In [ ]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

report_columns = ["Product",
                  "Model Name",
                  "Close",
                  "Target",
                  "Predicted",
                  "MSE",
                  "Percent",
                  "250Fees",
                  "5kFees",
                  "10kFees",
                  "250Profit",
                  "5kProfit",
                  "10k0Profit",
                  "Period"]

# we have a desired budget of 10k in trading capital to deploy. 
# "volValue": is the 24h total, the trading volume in quote currency of last 24 hours
# Any markets we enter need to have a signifcantly higher trading value volume otherwise
# we can't really trade there without breaking things up. As we grow our strat here will need to change.
# for now we cut down our set to 1m in volVal
volumeCutoff = 500000
df_products["volValue"] = [float(x) for x in df_products['volValue']]
df_products = df_products[df_products["volValue"] > volumeCutoff]
df_products = df_products.sort_values(by = ['id'])

models_in_play = [
        "lstm_cv", 
        "lstm_att_cv", 
        "lstm_cv_rvi",
        "svm_cv",
        "lstm_xgb_cols"]
if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
df_estc = pd.DataFrame(); #expected short term closes
df_estc["Product"] = [];
df_estc["Est Close"] = [];
df_estc["Est Close Raw"] = [];
bars_long = 180
bars_short = 30
counter = 0;
print("iterating through:", len(df_products))
tries = 3

def downloadAndPredict(all_trades, product, length):
  print("download day bars: ", product, " bar set:", length)
  df_raw = get_coin_data_frames(length, product)
  for name in models_in_play:
    print("predicting trade:", name, "for", product)
    df_trade = predict_config_model_for_product(df_raw, name, product)
    df_trade["Period"] = [length]
    df_trade = df_trade[report_columns]
    print("predicted mse:", name, ":{:.10f}".format(df_trade["MSE"].iloc[0]))
    all_trades = all_trades.append(df_trade)
  return all_trades

def downloadAndPredict15(all_trades, product):
  print("download 15 bars: ", product)
  [predicted_scaled, predicted] = fetch_and_predict_short_term(lstm_15m,product)
  df2 = pd.DataFrame({'Product': [product], 'Est Close': [predicted], 'Est Close Raw': predicted_scaled})
  all_trades = all_trades.append(df2)
  return all_trades


for index, row in df_products.iterrows():
    
    loop = True
    count = 0
    while(loop):
      try:
        print("start day long")
        df_trades = downloadAndPredict(df_trades, row.id, bars_long)
        time.sleep(1)
        
        print("start day short")
        df_trades = downloadAndPredict(df_trades, row.id, bars_short)
        time.sleep(1)

        print("start 15m")
        df_estc = downloadAndPredict15(df_estc, row.id)
        time.sleep(1)
        loop = False
      except Exception as inst:
        #raise inst
        print("Error: ", inst)
        time.sleep(1)
        count = count+1
        if count>tries:
          loop = False

df_trades.reset_index()
df_trades_filtered = df_trades  # TODO: Filter again after see what the new mse's look like #df_trades.loc[(df_trades["Period"] == 180) & (df_trades["MSE"] < 0.00005) | (df_trades["Period"] == 30) & (df_trades["MSE"] < 0.0005)]
df_buys = df_trades_filtered[df_trades_filtered['Percent'] > 0] 
df_shorts = df_trades_filtered[df_trades_filtered['Percent'] < 0] 
df_weighted = df_trades_filtered.groupby("Product").apply(consensus_prediction)
df_weighted = df_weighted.rename("Consensus Prediction")
df_trades_final = pd.merge(df_trades_filtered, df_weighted, left_on='Product', right_index=True)
df_trades_final["Consensus Percent"] = ((df_trades_final['Consensus Prediction'] - df_trades_final['Close']) / df_trades_final['Close']) * 100
df_view = df_trades_final[["Product",  "Model Name",  "MSE", "Period", "Close", "Predicted", "Percent","Consensus Percent", "Consensus Prediction"]]
df_view = df_view.sort_values(by=["Consensus Percent", "MSE"], ascending=[True, False])

Streaming output truncated to the last 5000 lines.
predicted mse: lstm_xgb_cols :0.0009605834
start day short
download day bars:  ADA-USDT  bar set: 30
predicting trade: lstm_cv for ADA-USDT
1/1 [==============================] - 0s 16ms/step
predicted mse: lstm_cv :0.0015713049
predicting trade: lstm_att_cv for ADA-USDT
1/1 [==============================] - 0s 16ms/step
predicted mse: lstm_att_cv :0.0021464989
predicting trade: lstm_cv_rvi for ADA-USDT
1/1 [==============================] - 0s 18ms/step
predicted mse: lstm_cv_rvi :0.0031912608
predicting trade: svm_cv for ADA-USDT
1/1 [==============================] - 0s 16ms/step
predicted mse: svm_cv :0.0001938703
predicting trade: lstm_xgb_cols for ADA-USDT
1/1 [==============================] - 0s 15ms/step
predicted mse: lstm_xgb_cols :0.0027734663
start 15m
download 15 bars:  ADA-USDT
1/1 [==============================] - 0s 17ms/step
start day long
download day bars:  AGIX-USDT  bar set: 180
predicting trade: lstm_cv for AGI

In [ ]:

df_view

,Product,Model Name,MSE,Period,Close,Predicted,Percent,Consensus Percent,Consensus Prediction
Date,,,,,,,,,
2023-01-23,GMT-USDT,lstm_xgb_cols,0.003068,30,0.644900,0.473397,-26.593805,-12.503618,0.564264
2023-01-23,GMT-USDT,lstm_cv_rvi,0.001332,30,0.644900,0.459325,-28.775825,-12.503618,0.564264
2023-01-23,GMT-USDT,lstm_att_cv,0.000951,30,0.644900,0.470085,-27.107317,-12.503618,0.564264
2023-01-23,GMT-USDT,lstm_cv,0.000728,30,0.644900,0.476030,-26.185501,-12.503618,0.564264
2023-01-23,GMT-USDT,svm_cv,0.000048,30,0.644900,0.511678,-20.657774,-12.503618,0.564264
...,...,...,...,...,...,...,...,...,...
2023-01-23,AXS3S-USDT,svm_cv,0.000757,30,0.000782,0.001110,41.920676,99.099710,0.001557
2023-01-23,AXS3S-USDT,lstm_cv_rvi,0.000381,180,0.000782,0.002087,166.849753,99.099710,0.001557
2023-01-23,AXS3S-USDT,lstm_att_cv,0.000266,180,0.000782,0.001672,113.792878,99.099710,0.001557


In [ ]:
df_estc

,Product,Est Close,Est Close Raw
0,AAVE-USDT,86.322760,0.098739
0,ACE-USDT,0.007918,0.258262
0,ADA-USDT,0.375038,0.253234
0,AGIX-USDT,0.180133,0.207015
0,ALBT-USDT,0.103957,0.232799
...,...,...,...
0,XPR-USDT,0.002053,0.229837
0,XRP-USDT,0.414237,0.265581
0,YGG-USDT,0.286585,0.186593
0,ZEC-USDT,47.856496,0.253045


In [ ]:
now = datetime.now()
today = now.strftime("%Y-%m-%d")

if coin_base:
  token = "cb"
else:
  token = "ku"

df_buys.to_csv(data_path+"/buy-" + token + "-" + today + ".csv")
df_estc.to_csv(data_path+"/15m-" + token + "-" + today + ".csv")
df_shorts.to_csv(data_path+"/shorts-" + token + "-" + today + ".csv")
df_trades_final.to_csv(data_path+"/final-" + token + "-" + today + ".csv")
df_view.to_csv(data_path+"/view-" + token + "-" + today + ".csv")